# ADA Project - Food self-sufficiency :   what about Switzerland?

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import folium

## We will first investigate the dataset we choose from the proposed list : "Global Food & Agriculture Statistics"

Based on this dataset containing informations about crops production for all countries over the world, our main idea was to link food production to hunger in some areas. One othr idea was to found the possible causes for food insufficiency (natural disasters, wars...) . 

## Load data  into a Pandas dataframe

In [ ]:
complete_dataset = pd.read_csv('../data/fao_data_crops_data.csv')

In [ ]:
# We split the data and metadata and store them in 'crops' and 'flags' dataframe, respecitvely.
crops = complete_dataset.loc[:2255342].copy() 
flags = complete_dataset.loc[2255344:2255348].copy() 
# 'flags' contains correspondance list of acronyms that describe how a given sample was acquired --> only informative
flags.drop(['element','year','unit','value','value_footnotes','category'], axis=1, inplace = True) 
flags.rename(columns={'country_or_area':'acronym', 'element_code':'description'}, inplace=True) 
flags.set_index('acronym', inplace=True)
flags

## Exploratory data analysis

In [ ]:
crops.head()

In [ ]:
print("Size of the DataFrame: {s}\n".format(s=crops.shape))
print("Variable types present in DataFrame: \n{t}".format(t=crops.dtypes))

In [ ]:
# List all the different footnotes values present in the dataset
footnotes = crops['value_footnotes'].unique() 
print(footnotes)
# Display dataframe that only contains one given value of 'value_footnotes'
display(crops.query('value_footnotes==@footnotes[4]')) 
# Return dataframe that only contains samples having NaN as value for 'value_footnotes'
crops[crops.value_footnotes.isnull()] 

In [ ]:
print(crops['element'].unique())
print(crops['year'].unique())
print(crops['unit'].unique())
print(crops['category'].unique())
print(crops['element_code'].unique())
print(crops['country_or_area'].unique())

## Data preprocessing

We clear the data by dropping all the row containing only NAN values. 
We also clear the raw where value_footnotes is NR as it means not repported by country, so it won't be usefull for our analysis. 

In [ ]:
# Returns a boolean of whether a column contains NaN (True) or not (False).
print(crops.isnull().values.any(axis=0)) 

# Drop rows which contain only missing values.
crops.dropna(how='all', inplace=True) 

In [ ]:
# We drop the samples where 'value' is unknown (NaN) because they are of no utility    
crops.dropna(subset=['value', 'value_footnotes'], inplace=True) 

# Let's drop also all the samples that have 'NR' as a 'value_footnotes' value or 0 as 'value'
crops.drop(index=crops[crops['value_footnotes'].str.contains('NR')].index, inplace=True)
crops.drop(index=crops[crops['value']==0].index, inplace=True)


In our dataset, regions are indicated by a "+" at the end of their names. We want to separate regions from countries to facilitate our analysis so we can be more precise. 

In [ ]:
regions_bool = crops['country_or_area'].str.contains('\+')
crops_regions = crops[regions_bool].copy()
crops_countries = crops[~regions_bool].copy()
crops_countries[crops_countries.country_or_area.str.contains('China')].tail()

We calculate the mean of all the elements for every country so we can compare the area harvested, seed or yield between each country. The mean is calculated over all years. 

In [ ]:
#calculate the mean of all the elements for every country.  
crops_countries_by_country_year = crops_countries.groupby(['country_or_area', 'element']) \
                            .agg({'value':'mean'}) \
                            .rename(columns={'value':'mean_'}) 
crops_countries_by_country_year

In [ ]:
area_harvested = crops_countries_by_country_year.loc[(slice(None),'Area Harvested'), :]
area_harvested.loc['United States of America']

#### Create a map showing yield by country (average over all years) 

Even if we are not sure to keep those following maps, we made them to gt familiar with the data, moreover, it gives us an insight of the most important countries in terms of food production or area harvested.

In [ ]:
yield_df= crops_countries_by_country_year.loc[(slice(None),'Yield'), :]
#we take the log value for the following plot, so our quantile a more equilibrate
log_yield_df=pd.DataFrame(yield_df.mean_.map(lambda x:np.log(x)))
log_yield_df.head()

In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3)

world_geo = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'
Bins = list(yield_df.mean_.quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(zoom_start=3)

folium.Choropleth(
    geo_data=world_geo,
    name='choropleth',
    data=log_yield_df,
    columns=[log_yield_df.index.get_level_values(level='country_or_area').values,'mean_'],
    key_on='feature.properties.name',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='yield',
    #bins = Bins,
    reset=True
).add_to(m)

folium.LayerControl().add_to(m)

m

The countries having the most important yield are Iceland and Danemark. We can also see that in many countries of Africa the yield is very low. Mongolia has also a very low yield.

#### Area Hervested (mean)/Area Hervested (2018) by country

Such a way to calculate the ratio is hard o interpret. What we should do instead is to calculate an average for each year and make an interactive plot so we can select the year we want to analyze and show the map. 

In [ ]:
surface_country = pd.read_csv('../data/API_AG.LND.TOTL.K2_DS2_en_csv_v2_422954.csv', skiprows=3)
surface_country.set_index('Country Name', inplace = True)
crops_countries_area = area_harvested.join(surface_country['2018'], on='country_or_area', how='left')
crops_countries_area['ratio'] = area_harvested['mean_']/(crops_countries_area['2018']*100)
crops_countries_area.dropna(inplace=True)

pl = crops_countries_area.plot(kind="bar", 
                             y="ratio", 
                            figsize=(30, 7), alpha=0.5, color="olive")

In [ ]:
crops_countries_area.head()

#### Create a map showing this ratio by country

In [ ]:
crops_countries_area_df=pd.DataFrame(crops_countries_area.ratio)
crops_countries_area_df.head()
log_df=pd.DataFrame(crops_countries_area_df.ratio.map(lambda x:np.log(x)))
type(log_df)

In [ ]:
m = folium.Map(location=[48, -102], zoom_start=3)

world_geo = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'
Bins = list(crops_countries_area.ratio.quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(zoom_start=3)

folium.Choropleth(
    geo_data=world_geo,
    name='choropleth',
    data=log_df,
    columns=[crops_countries_area.index.get_level_values(level='country_or_area').values,'ratio'],
    key_on='feature.properties.name',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='country surface vs surface harvested ratio',
    #bins = Bins,
    reset=True
).add_to(m)

folium.LayerControl().add_to(m)

m

In [ ]:
crops_countries_by_country_by_category = crops_countries.groupby(['country_or_area','element', 'category'])
crops_countries_by_country_by_category = pd.DataFrame(data=crops_countries_by_country_by_category.value.sum().reset_index(name='total').sort_values(by='total',ascending=False))

crops_countries_by_country_by_category.head(10)

we can see that China and the united states are the main producers of cereals_total and cereale_rice_milled_eq. 

  
 ## What are the principal foodstuffs produced in each country/region of the world? And which countries are the biggest producers for a given food?

In [ ]:
main_product=crops_countries_by_country_by_category.drop_duplicates(subset='country_or_area', keep='first')
main_product.head(10)

We decide to keep only the Production quantity for further analysis.

In [ ]:
ind_keep=pd.Series(main_product.element.str.contains('Production Quantity'))
ind_keep=ind_keep[ind_keep].index

main_product_quantity = main_product.drop(index=main_product.index.difference(ind_keep))
main_product_quantity.head(10)

In [ ]:
main_product_quantity.category.unique()

In the previous dataframe (main_product_quantity) we show the category that is most produced by each country in term of production quantity. 

We can see that China and the United States are the main producers of cereals, Canada is the main producer of cereals_rice_milled. Nigeria and Poland are the mais producers of roots and tubbers. Philippines mainly produces sugar cane and Malesia mainly produces oil_palm_fruit. 

## Are all countries equal in terms of diversity of foodstuffs harvested?
    
To answer this question, we simply count the number of categories produced by countries, so we can get an idea of their food production diversity.

In [ ]:
food_diversity = pd.DataFrame(crops_countries.groupby(['country_or_area','category'])['category'].count().reset_index(name='total'))
food_diversity.head(10)

In [ ]:
food_diversity = pd.DataFrame(food_diversity.groupby(['country_or_area']).country_or_area.size().reset_index(name='category_diversity'))

In [ ]:
food_diversity.sort_values('category_diversity', ascending=False).head(10)

Should we combine China and China, mainland?? --> We could keep only China for further analysis.

## Interactive visualization of categories by year and by element

Categories are the different products.
Element corresponds to yield, harvested area, production quantity. 

#### Interactive visualization plot #1


In [ ]:
#TO RUN THIS: with conda --> conda install -c conda-forge ipywidgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
#To enable interactive viz on lab --> conda install nodejs
#                                  + jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
#Interactive visualization
def viz_year(year, element, category):
    df_to_plot = crops_regions.loc[crops_regions['year'] == year].loc[crops_regions['element'] == element].loc[crops_regions['category'] == category]
    df_to_plot.plot(kind = 'bar', x='country_or_area', y='value',figsize=(20,10))
    plt.title(f'{element} values of {category} by regions for year {year}', fontsize=20)
    plt.xlabel("Regions", fontsize= 20)
    plt.ylabel("Values", fontsize= 20)
    plt.show()

years = crops_regions.year.unique()    
elements = crops_regions.element.unique()
categories = crops_regions.category.unique()
interact(viz_year, year=years, element = elements , category=categories)    

We can see that for some categories there are no numerical values to plot --> Remove them from dataframe? --> only for some categories (eg cereals_total)


### Interactive visualization of element of a given category by a given country throughout years

In [ ]:
#Interactive visualization
def viz_evolution(country, element, category):
    df_to_plot = crops_countries.loc[crops_countries['country_or_area'] == country].loc[crops_countries['element'] == element].loc[crops_countries['category'] == category]
    df_to_plot.plot(x='year', y='value',figsize=(20,10))
    plt.title(f'{element} values of {category} in {country} throughout years', fontsize= 20)
    plt.xlabel("Year", fontsize= 20)
    plt.ylabel("Values", fontsize= 20)
    plt.show()

countries = crops_countries.country_or_area.unique()    
elements = crops_countries.element.unique()
categories = crops_countries.category.unique()
interact(viz_evolution, country=countries, element = elements , category=categories)    

Now that we have well understood our dataset we can redefine our project goals. 
As we have many datas for allmost all the country in the words we had to redefine the direction we wanted to follow. So we decided to focus our attention on one country only: Switzerland!
We will try to answer the following question: Can Switzerland be self-sufficient in term of food production.
see updates on the Readme and project_ch.ipynb

# Project Update - Insight into Switzerland agriculture production 

We will focus on Switzerland compared to its neighbours. We would like to know if Switzerland could be self-sufficient in term of food production. 

## Abstract

In the wake of the the years 2007-08, food self-sufficiency policies have gained increased attention in a number of coutries following the international food crisis that triggered great volatilities on the world food markets causing important economic and social damages. <br>
Since then, diverse countries have expressed interest in improving their levels of food self-sufficiency arising controversy into a massive economically connected world.

On the 23th september of 2018, in the small country of Switzerland, the debate is materialized into a popular referendum submitted to its population asking wherever a food self-sufficiency politic should be adopted or not. Such a politic could have unexpected consequences considering a country as Switzerland with many neighbours and such a small area capacity. <br>
This paper aims to analyse the questions surrounding the debate over food self-sufficiency in Switzerland. 

- What does Switzerland produce and in which quantity? 
- What about the amount of importations/exportations?
- Are all Switzerland areas optimally harvested?
- Links to population size
- How is the Swiss productiviy trend evolving, is it correlated with external factors such as temperature, fertilizer use, ...?

Then we will make comparaisons between Switzerland and its neighbours. Does CH import more than its neighbours (due to its small size ?) ? Is food self-suffience of CH realistic ? How many farms/farmers would it need ?


## Data loading - Crops 

This dataset represents our new start line, it contains almost the same informations as the "Global Food & Agriculture Statistics" we already used. However, the data are more recent.  

We found the majority of our following data on the __[Food And Agriculture Organization of the United Nations Datasets](http://www.fao.org/faostat/en/#data)__ website (we will precise later if one dataset does not come from this link).

File contains data about Switzerland and neighbours (Italy, Germany, France, Austria and Liechtenstein)

In [ ]:
raw_CH_crops_dataset = pd.read_csv('../data/FAOSTAT_data_crops_CHandNeighbours.csv')

Let's explore the structure of our dataset :

In [ ]:
raw_CH_crops_dataset.head()

Keep only relevant information.

In [ ]:
raw_CH_crops_dataset =raw_CH_crops_dataset[['Domain', 'Area', 'Element', 'Item', 'Year', 'Unit', 'Value', 'Flag Description']]

In [ ]:
raw_CH_crops_dataset.drop(index=raw_CH_crops_dataset[raw_CH_crops_dataset['Flag Description'].str.contains('Data not available')].index, inplace=True)

In [ ]:
raw_CH_crops_dataset.head()

Let's also load the flags dataset, in case we need it later (very small size --> doesn't cost anything)

In [ ]:
flags = pd.read_csv('../data/FAOSTAT_data_flags.csv')
flags

In [ ]:
print("Size of the DataFrame: {s}\n".format(s=raw_CH_crops_dataset.shape))
print("Variable types present in DataFrame: \n{t}".format(t=raw_CH_crops_dataset.dtypes))

Null values investigation:

In [ ]:
print(raw_CH_crops_dataset.isnull().values.any(axis=0)) 

No NaN values found. Perfect.

What about the categories listed in our columns?

In [ ]:
print(raw_CH_crops_dataset['Domain'].unique())
print(raw_CH_crops_dataset['Area'].unique())
print(raw_CH_crops_dataset['Element'].unique())
print(raw_CH_crops_dataset['Item'].unique())
print(raw_CH_crops_dataset['Year'].unique())
print(raw_CH_crops_dataset['Unit'].unique())
print(raw_CH_crops_dataset['Flag Description'].unique())

**Quick view of the crops dataset ready to be used**

In [ ]:
raw_CH_crops_dataset.head()

## **Crops plots :** what we can already see/investigate with this first dataset

Even if we will probably not use those plots for final presentation/analysis, it helps us to see what's inside our data. It is quick and very visual.

### Plot production of all countries over time for a selected crop

This plot is interactive. It allows you to choose for an item (apples, berries..) and shows you its production over years for the 6 countries (CH + Neighbours as listed above).

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
# All those  library importations would probably be at the begining for the final version of te notebook, 
# but for now, we just kee them where we use them since we don't know what we will keep or not.

In [ ]:
#Interactive visualization

#Plot the production of selected item for all countries over years
def viz_evolution(item):
    df_viz_evolution = raw_CH_crops_dataset.loc[raw_CH_crops_dataset['Element']=='Production'].loc[raw_CH_crops_dataset['Item']==item]
    
    # multiple line plot
    plt.figure(figsize=(20,10))
    plt.plot( 'Year', 'Value', data=df_viz_evolution.loc[df_viz_evolution['Area']=='Austria'], marker='', color='green',  label = 'Austria')
    plt.plot( 'Year', 'Value', data=df_viz_evolution.loc[df_viz_evolution['Area']=='France'], marker='', color='skyblue', label = 'France')
    plt.plot( 'Year', 'Value', data=df_viz_evolution.loc[df_viz_evolution['Area']=='Switzerland'], marker='', color='red', label = 'Switzerland', linewidth=3)
    plt.plot( 'Year', 'Value', data=df_viz_evolution.loc[df_viz_evolution['Area']=='Germany'], marker='', color='orange', label = 'Germany')
    plt.plot( 'Year', 'Value', data=df_viz_evolution.loc[df_viz_evolution['Area']=='Italy'], marker='', color='grey', label = 'Italy')
    
    plt.legend() 
    plt.title(f'Production of {item} in Switzerland and its neighbours throughout years', fontsize= 20)
    plt.xlabel("Year", fontsize= 20)
    plt.ylabel("Values", fontsize= 20)
    plt.show()
   
items = raw_CH_crops_dataset.Item.unique()
interact(viz_evolution, item = items)    

For some datas we can see that we miss datas along some years. 
ex: item=Artichokes
What do we do? linear interpolation? drop this item? 

### Plot production/area_harvested for all items of all countries over time.

This plot is interactive. It allows you to choose for an element (production/area harvested/yield) and shows you the sum of all items for each country over years (CH & Co)

In [ ]:
# plot area harvested of each country over years
crops_sum = raw_CH_crops_dataset.groupby(['Area', 'Element','Year']) \
                                .agg({'Value':'sum'}) \
                                .rename(columns={'Value':'Sum'}) \
                                .reset_index()
crops_sum.head() # Sum of area/yiel/production of items by country and year

In [ ]:
#Interactive visualization

#Plot the area harvested (sum of all items) for all countries over years
def viz_sum_evolution(element):
    df_viz_sum_evolution = crops_sum.loc[crops_sum['Element']== element]
    
    # multiple line plot
    plt.figure(figsize=(20,10))
    plt.plot( 'Year', 'Sum', data=df_viz_sum_evolution.loc[df_viz_sum_evolution['Area']=='Austria'], marker='', color='green',  label = 'Austria')
    plt.plot( 'Year', 'Sum', data=df_viz_sum_evolution.loc[df_viz_sum_evolution['Area']=='France'], marker='', color='skyblue', label = 'France')
    plt.plot( 'Year', 'Sum', data=df_viz_sum_evolution.loc[df_viz_sum_evolution['Area']=='Switzerland'], marker='', color='red', label = 'Switzerland', linewidth=3)
    plt.plot( 'Year', 'Sum', data=df_viz_sum_evolution.loc[df_viz_sum_evolution['Area']=='Germany'], marker='', color='orange', label = 'Germany')
    plt.plot( 'Year', 'Sum', data=df_viz_sum_evolution.loc[df_viz_sum_evolution['Area']=='Italy'], marker='', color='grey', label = 'Italy')
    
    plt.legend() 
    plt.title(f'{element} of all items in Switzerland and its neighbours throughout years', fontsize= 20)
    plt.xlabel("Year", fontsize= 20)
    plt.ylabel("Values", fontsize= 20)
    plt.show()
   
elements = crops_sum.Element.unique()
interact(viz_sum_evolution, element = elements)  

## Data loading - Land use indicators 

File contains data about Switzerland and neighbours (Italy, Germany, France, Austria and Liechtenstein).
This file will allows us to know the potential of Switzerland in term of agriculture. Does the country use all its land or not? 

Data exploration and pre-processing is very simmilar to first dataset. We will therefore not describe all steps as precisely as before.

In [ ]:
raw_land_use_dataset = pd.read_csv('../data/FAOSTAT_data_LandUseIndicators.csv')

In [ ]:
raw_land_use_dataset.head()

In [ ]:
raw_land_use_dataset =raw_land_use_dataset[['Domain', 'Area', 'Element', 'Item', 'Year', 'Unit', 'Value', 'Flag Description']]

In [ ]:
print("Size of the DataFrame: {s}\n".format(s=raw_land_use_dataset.shape))
print("Variable types present in DataFrame: \n{t}".format(t=raw_land_use_dataset.dtypes))

In [ ]:
print(raw_land_use_dataset.isnull().values.any(axis=0))  # --> PERFECT!

In [ ]:
print(raw_land_use_dataset['Domain'].unique())
print(raw_land_use_dataset['Area'].unique())
print(raw_land_use_dataset['Element'].unique())
print(raw_land_use_dataset['Item'].unique())
print(raw_land_use_dataset['Year'].unique())
print(raw_land_use_dataset['Unit'].unique())
print(raw_land_use_dataset['Flag Description'].unique())

## **Land use indicators plots :** what we can already see/investigate with this second dataset

Even if we will probably not use those plots for final presentation/analysis, it helps us to see what's inside our data. It is quick and very visual.

### Plot the lands distribution in Switzerland

We would like to precise those data (with more datasets) by including also urban areas to the distribution.

In [ ]:
import matplotlib.pyplot as plt

# DataFrames to plot
df_land = raw_land_use_dataset.loc[raw_land_use_dataset['Area']=='Switzerland'].loc[raw_land_use_dataset['Year']==2016].loc[raw_land_use_dataset['Element']=='Share in Land area']
df_agri = raw_land_use_dataset.loc[raw_land_use_dataset['Area']=='Switzerland'].loc[raw_land_use_dataset['Year']==2016].loc[raw_land_use_dataset['Element']=='Share in Agricultural land']

# Pie plot #1
labels1 = df_land.Item
sizes1 = df_land.Value
explode = (0, 0, 0.1, 0)  # only "explode" the 3rd slice

fig1, ax1 = plt.subplots()
ax1.pie(sizes1, explode=explode,labels=labels1, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax1.title.set_text('Distribution of lands in Switzerland, year 2016')
fig1.set_facecolor('white')

# Pie plot #2
labels2 = df_agri.Item
sizes2 = df_agri.Value
fig1, ax2 = plt.subplots()
ax2.pie(sizes2, labels=labels2, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax2.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
ax2.title.set_text('Distribution of agricultural lands in Switzerland, year 2016')

# General settings
fig1.set_facecolor('white')
plt.show()

## Data loading - Demographical data 

File contains data about Switzerland and neighbours (Italy, Germany, France, Austria but not Liechtenstein -data missing from dataset).
These data will allow us to know the number of consumers in Switzerland and to compare the possible food self-sufficiency between Switzerland and its neighbours. we would like to answer questions as: With the growing population, can we feed everybody with Swiss agriculture in the next few years?

Data exploration and pre-processing is very simmilar to first dataset. We will therefore not describe all steps as precisely as before.

In [ ]:
demography = pd.read_csv('../data/FAOSTAT_data_demography.csv')

In [ ]:
demography

In [ ]:
for col in demography:
    print (demography[col].unique())

In [ ]:
demography = demography[['Area', 'Year', 'Value']]
demography

The value unit beeing 1000 persons, we adjust the number to display the population values in terms of individuals

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
demography["Population"] = demography.Value.apply(lambda x: x*1000)
demography.drop(columns='Value')

In [ ]:
#plot of the evolution of the demography over the years
plt.figure(figsize=(20,10))
plt.plot( 'Year', 'Population', data=demography.loc[demography['Area']=='Austria'], marker='', color='green',  label = 'Austria')
plt.plot( 'Year', 'Population', data=demography.loc[demography['Area']=='France'], marker='', color='skyblue', label = 'France')
plt.plot( 'Year', 'Population', data=demography.loc[demography['Area']=='Switzerland'], marker='', color='red', label = 'Switzerland', linewidth=3)
plt.plot( 'Year', 'Population', data=demography.loc[demography['Area']=='Germany'], marker='', color='orange', label = 'Germany')
plt.plot( 'Year', 'Population', data=demography.loc[demography['Area']=='Italy'], marker='', color='grey', label = 'Italy')
    
plt.legend() 
plt.title('Evolution of the demography over the years' , fontsize= 20)
plt.xlabel("Year", fontsize= 20)
plt.ylabel("Population value", fontsize= 20)
plt.show()

We can see that as expected, the population is growing in every country. 

## Data loading - Swiss importations and exportations of agricultural goods 

Files contain data for Switzerland only. The data provide insight about the trade of agricultural goods, that is the importations and exportations for a given product.
Data exploration and pre-processing is very simmilar to first dataset. We will therefore not describe all steps as precisely as before.

In [ ]:
CH_imports = pd.read_csv('../data/FAOSTAT_data_11-23-2019.csv')

In [ ]:
CH_imports.head()

In [ ]:
CH_exports = pd.read_csv('../data/FAOSTAT_data_exports.csv')

In [ ]:
CH_exports.head()

To make the data processing and analysis more simple and concise, we concatenate exportations and importations data (as both datasets have exactly the same structure).

In [ ]:
CH_trade = pd.concat([CH_imports, CH_exports])

In [ ]:
CH_trade.dtypes

In [ ]:
for col in CH_trade:
    print (CH_trade[col].unique())

For maximizing reliability of later results, we discard the numbers that were obtained from an unofficial source.

In [ ]:
unofficial_stats_index = CH_trade.loc[CH_trade.Flag=='*'].index

In [ ]:
# Drop the unofficial data
CH_trade = CH_trade.drop(index = unofficial_stats_index)

We keep only the importation and exportation values that are represented in tonnes, so that we can compare it with the agricultural production.

In [ ]:
CH_trade = CH_trade.loc[CH_imports.Unit=='tonnes']

In [ ]:
CH_trade = CH_trade[['Element','Partner Countries', 'Item', 'Year', 'Unit', 'Value']]

To keep the model simple, we sum the importations and exportations for a given product over all partner countries.

In [ ]:
CH_trade = CH_trade.groupby(['Item', 'Year', 'Element']).agg({'Value':'sum'})\
                                    .reset_index()

We improve the structure of our dataframe by pivoting its values of importations and exportations.

In [ ]:
CH_trade_transformed = pd.pivot(CH_trade,columns = 'Element', values='Value')\
                .rename(columns={'Export Quantity':'Exported Quantity','Import Quantity':'Imported Quantity'})

In [ ]:
CH_trade_transformed

In [ ]:
CH_trade = pd.concat([CH_trade, CH_trade_transformed], axis=1, join='inner')

In [ ]:
CH_trade.drop(columns=['Value', 'Element'], inplace=True)

In [ ]:
CH_trade = CH_trade.groupby(['Item', 'Year'])\
                            .agg({'Exported Quantity':'mean','Imported Quantity':'mean'})\
                            .reset_index()
                                    

Combine production and trade data in one dataframe 'CH_data' so that we have all the information at the same place. Note that we don't have values of importations and exportations before 1986 so production of goods before 1986 will not be considered as from here.

In [ ]:
CH_crops = raw_CH_crops_dataset[['Area', 'Item','Element', 'Year', 'Unit', 'Value']]

In [ ]:
# Merge importations data with production data
CH_data = CH_crops.loc[CH_crops.Area=='Switzerland'].loc[CH_crops.Element=='Production'].loc[CH_crops.Year>= 1986]\
                                    .merge(CH_trade,on=['Item', 'Year'], how='inner')\
                                    .rename(columns={'Value':'Produced Quantity'})



In [ ]:
CH_data

Now, combine with the land analysis of Switzerland, the consumers trends and the Swiss demography we could estimate if the country has an interest of producing more of an item, if it is able to produce more of an item and stop its importation of the item. --> # Milestone 3

### Plot production, exports and imports of items in Switzerland over years.

This plot is interactive. It allows you to choose for an item (apples, berries..) and shows you its production, exportation and importation in Switzerland over years.

In [ ]:
#Interactive visualization

#Plot the production of selected item for all countries over years
def viz_evolution(item):
    df_viz_evolution = CH_data.loc[CH_data['Item']==item]
    
    # multiple line plot
    plt.figure(figsize=(20,10))
    plt.plot( 'Year', 'Produced Quantity', data=df_viz_evolution, marker='', color='red', label = 'crops', linewidth=3)
    plt.plot('Year', 'Imported Quantity', data=df_viz_evolution, marker='', color='blue', label = 'imports', linewidth=3)
    plt.plot('Year', 'Exported Quantity', data=df_viz_evolution, marker='', color='green', label = 'exports', linewidth=3) 
    plt.legend() 
    plt.title(f'Production and imports of {item} in Switzerland throughout years', fontsize= 20)
    plt.xlabel("Year", fontsize= 20)
    plt.ylabel("Values [tonnes]", fontsize= 20)
    plt.show()
   
items = CH_data.Item.unique()
interact(viz_evolution, item = items)    

**Most produced, imported and exported products :**

- Most produced crops products

In [ ]:
CH_data.loc[CH_data.Year == 2016].sort_values(by='Produced Quantity', ascending = False).head(10)

- Most imported crops products

In [ ]:
CH_data.loc[CH_data.Year == 2016].sort_values(by='Imported Quantity', ascending = False).head(10)

- Most exported crops products

In [ ]:
CH_data.loc[CH_data.Year == 2016].sort_values(by='Exported Quantity', ascending = False).head(10)

### Plot most produced, exported and  imported items in Switzerland, year 2016.

This plot is interactive. Shows values upon cursor selection.

In [ ]:
import plotly # conda install -c anaconda plotly #AND# jupyter labextension install @jupyterlab/plotly-extension
import plotly.graph_objects as go
y_wheat = CH_data.loc[CH_data.Year == 2016].loc[CH_data.Item=='Wheat'].values[0,-3:]
y_potatoes = CH_data.loc[CH_data.Year == 2016].loc[CH_data.Item=='Potatoes'].values[0,-3:]
y_beet = CH_data.loc[CH_data.Year == 2016].loc[CH_data.Item=='Sugar beet'].values[0,-3:]
y_maize = CH_data.loc[CH_data.Year == 2016].loc[CH_data.Item=='Maize'].values[0,-3:]

x=['Produced', 'Exported', 'Imported']
fig = go.Figure(go.Bar(x=x, y=y_wheat, name='Wheat'))
fig.add_trace(go.Bar(x=x, y=y_potatoes, name='Potatoes'))
fig.add_trace(go.Bar(x=x, y=y_beet, name='Sugar beet'))
fig.add_trace(go.Bar(x=x, y=y_maize, name='Maize'))

fig.update_layout(
    title='Most produced, exported and imported items in Switzerland in 2016',
    yaxis_title="Values [tonnes]",
    barmode='stack', 
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#7f7f7f")
    )
fig.show()


We see that Switzerland is a very small exportator. 

### Plot production, importation and exportation of agricultural goods in Switzerland throughout years

This plot is interactive. Shows values upon cursor selection

In [ ]:
import plotly.graph_objects as go

def viz_potatoe(item):
    y_wheat = CH_data.loc[CH_data.Item==item].values[:,-3:]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=CH_data.Year.unique(), y=y_wheat[:,0], fill='tonexty', name='Produced')) # fill down to xaxis
    fig.add_trace(go.Scatter(x=CH_data.Year.unique(), y=y_wheat[:,1], fill='tozeroy', name='Exported')) # fill to trace0 y
    fig.add_trace(go.Scatter(x=CH_data.Year.unique(), y=y_wheat[:,2], fill='tonexty', name='Imported')) # fill to trace0 y
    fig.update_layout(
        title=f"{item} importations and productions throughout years in Switzerland",
        yaxis_title="Values [tonnes]",
        xaxis_title='Years'
        )
    fig.show()

items = CH_data.Item.unique()
interact(viz_potatoe, item = items)  


### Plot production,  importation and exportation in Switzerland throughout years

This plot is interactive. Shows values upon cursor selection. As reported before, values of exportations are much lower than those of production and importations. Hence, exportations values will now be plot separately, to better show their trend.

In [ ]:
total_crops_imports = CH_data.groupby('Year').agg({'Produced Quantity':'sum', 'Exported Quantity':'sum', 'Imported Quantity':'sum'})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=CH_data.Year.unique(), y=total_crops_imports['Produced Quantity'].values, fill='tonexty', name='Produced')) # fill down to xaxis
fig.add_trace(go.Scatter(x=CH_data.Year.unique(), y=total_crops_imports['Imported Quantity'].values, fill='tozeroy', name='Imported')) # fill to trace0 y
fig.update_layout(
    title="Sum of all importations and productions throughout years in Switzerland",
    yaxis_title="Values [tonnes]",
    xaxis_title='Years'
    )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=CH_data.Year.unique(), y=total_crops_imports['Exported Quantity'].values, fill='tozeroy', name='Exported')) # fill to trace0 y
fig.update_layout(
    title="Sum of exportations throughout years in Switzerland",
    yaxis_title="Values [tonnes]",
    xaxis_title='Years'
    )
fig.show()

Maybe we could add here an intercative plot where we can select the item to see (but for now we are sure if we can/how to combine plotly and ipwidgets libraries)

In [ ]:
CH_data2 = CH_data.copy().rename(columns={'Produced Quantity':'Country production', 'Imported Quantity':'Importation', 'Exported Quantity':'Exportation'})
CH_data_transformed = pd.melt(CH_data2, value_vars=['Country production', 'Importation'], id_vars=['Area', 'Element','Item','Year','Unit'], var_name='Input', value_name='Value')

In [ ]:
CH_data_transformed.loc[CH_data_transformed.Item=='Potatoes']

### Plot evolution of production and importations for five most important items (Switzerland data only)

This plot is interactive. Shows values upon cursor selection

In [ ]:
CH_restrained = CH_data_transformed.loc[CH_data_transformed.Item.isin(['Apples','Wheat','Potatoes', 'Maize', 'Oats'])]



In [ ]:
# Just trying a plot
import plotly.express as px
fig = px.area(CH_restrained, x="Year", y="Value", color='Item',
      line_group='Input')
fig.update_layout(
    title="Switzerland's production/importation evolution for five most important items",
    yaxis_title="Values [tonnes]",
    xaxis_title='Years'
    )
fig.show()

In [ ]:
CH_data_transformed_exportations = pd.melt(CH_data2, value_vars='Exportation', id_vars=['Area', 'Element','Item','Year','Unit'], var_name='Input', value_name='Value')


In [ ]:
CH_restrained_exportations = CH_data_transformed_exportations.loc[CH_data_transformed_exportations.Item.isin(['Apples','Wheat','Potatoes', 'Maize', 'Oats'])]

In [ ]:
import plotly.express as px
fig = px.area(CH_restrained_exportations, x="Year", y="Value", color='Item',
      line_group='Input')
fig.update_layout(
    title="Switzerland's exportations evolution for five most important items over time",
    yaxis_title="Values [tonnes]",
    xaxis_title='Years'
    )
fig.show()

## Data Loading -  Switzerland temperatures

This dataset does not come from FAOSTATS but from : __[MeteoSwiss](https://www.meteoswiss.admin.ch/home/climate/swiss-climate-in-detail/Swiss-temperature-mean/Data-on-the-Swiss-temperature-mean.html)__

In [ ]:
CH_temperatures = pd.read_csv('../data/10.18751-Climate-Timeseries-CHTM-1.1-swiss.txt', sep="\t", header=0, skiprows=15)

In [ ]:
CH_temperatures = CH_temperatures.loc[CH_temperatures.time>=1986].loc[CH_temperatures.time<=2017]

In [ ]:
CH_temperatures = CH_temperatures.iloc[:,-3:]

In [ ]:
CH_temperatures

### Plot : Is there a correlation between production and temperature?


In [ ]:
years = np.sort(CH_data.Year.unique())
fig, ax1 = plt.subplots()
data1 = CH_data.loc[CH_data.Item=='Potatoes']['Produced Quantity']
data2 = CH_temperatures.year

color = 'tab:red'
ax1.set_xlabel('year')
ax1.set_ylabel('production', color=color)
ax1.plot(years, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('temperature', color=color)  # we already handled the x-label with ax1
ax2.plot(years, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.title('Potatoes production and temperatures every year')
plt.show()

## Plan - What's coming next?

<div class="alert alert-block alert-info">
    
1. Defining what is food self-sufficiency
    1. $ SSR = Production * 100 / (Production + Imports - Exports)$ to develop
    2. Addapt it to the Swiss case : take a look to what we import (basic needs ?), export (top exports ? by far ?) and production graphs
    3. __[Ref. Paper "Food self-sufficiency: Making sense of it, and when it makes sense" By Jennifer Clapp](https://www.sciencedirect.com/science/article/pii/S0306919216305851#b0240)__. <br> Résumé : __[Résumé par le site Resilience du paper de Clapp](https://www.resilience.org/stories/2018-03-13/food-self-sufficiency-does-it-make-sense/)__
    4. Compare our results with other sources just to know if we share the same results (e.g. selfsufficiency switzerland on wikipedia __[List of countries by food self-sufficieent rate](https://en.wikipedia.org/wiki/List_of_countries_by_food_self-sufficiency_rate)__)

    
2. Food situation of Switzerland from 1986 to 2017.
    1. Is/was it food self-sufficient ? SSR scores over the years.
    2. Comapre to neighbours

    
3. Will it be **physically** possible for Switzerland in a near future to be food self-sufficient (in the sense of the 2018 initiative bc we have seen that definition is relative) taking into account its population growth (hesimated increase in consumption computation)? What would it imply/take into account in terms of :
    1. Area harvested (actual ratio and estimation of its evolution)
    2. Farmers population 
    3. Temperature (climate impact food production correlation)
    4. Environment (use of fertilizers needed ? depends on productivity)

    
4. Attempt on **economy** consequences analysis ?
    1. Complicated ... What about looking at what happended in countries that adopted food self-sufficient policies such as Senegal, India, the Philippines, Qatar, Bolivia, and Russia ? (Jaccard and correlations?)
    2. Jaccard similarity of country based on SSR to see which country should adopt more food self-sufficient policies ?